In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 0
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.19 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_1S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1


C:\Users\test\anaconda3\envs\pytorch\lib\site-packages\torchvision\ops\misc.py:120: UserWarning: Don't use ConvNormActivation directly, please use Conv2dNormActivation and Conv3dNormActivation instead.
  warnings.warn(
Downloading: "https://github.com/fschmid56/EfficientAT/releases/download/v0.0.1/mn40_as_ext_mAP_487.pt" to resources\mn40_as_ext_mAP_487.pt
9.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

37.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_w

Train Epoch: 1/50 Train_Loss: 2.3028810024261475 Train_acc: 8.500000%
Train Epoch: 2/50 Train_Loss: 2.273299217224121 Train_acc: 9.687500%
Train Epoch: 3/50 Train_Loss: 2.2891087532043457 Train_acc: 10.687500%
Train Epoch: 4/50 Train_Loss: 2.2861099243164062 Train_acc: 12.000000%
Train Epoch: 5/50 Train_Loss: 2.22670578956604 Train_acc: 15.750000%
Train Epoch: 6/50 Train_Loss: 2.1917788982391357 Train_acc: 21.937500%
Train Epoch: 7/50 Train_Loss: 2.0214855670928955 Train_acc: 39.625000%
Train Epoch: 8/50 Train_Loss: 1.7929394245147705 Train_acc: 59.187500%
Train Epoch: 9/50 Train_Loss: 1.4125118255615234 Train_acc: 74.125000%
Train Epoch: 10/50 Train_Loss: 1.19581139087677 Train_acc: 82.812500%
Train Epoch: 11/50 Train_Loss: 0.8696566820144653 Train_acc: 85.125000%
Train Epoch: 12/50 Train_Loss: 0.6089949011802673 Train_acc: 87.437500%
Train Epoch: 13/50 Train_Loss: 0.5786595940589905 Train_acc: 89.562500%
Train Epoch: 14/50 Train_Loss: 0.45803821086883545 Train_acc: 89.750000%
Train E

Train Epoch: 1/50 Train_Loss: 2.3562841415405273 Train_acc: 12.500000%
Train Epoch: 2/50 Train_Loss: 2.2650322914123535 Train_acc: 12.312500%
Train Epoch: 3/50 Train_Loss: 2.3256945610046387 Train_acc: 13.250000%
Train Epoch: 4/50 Train_Loss: 2.2804977893829346 Train_acc: 16.250000%
Train Epoch: 5/50 Train_Loss: 2.2061259746551514 Train_acc: 18.250000%
Train Epoch: 6/50 Train_Loss: 2.114480495452881 Train_acc: 27.375000%
Train Epoch: 7/50 Train_Loss: 1.9393367767333984 Train_acc: 41.875000%
Train Epoch: 8/50 Train_Loss: 1.7096058130264282 Train_acc: 60.875000%
Train Epoch: 9/50 Train_Loss: 1.4588512182235718 Train_acc: 76.062500%
Train Epoch: 10/50 Train_Loss: 1.0721529722213745 Train_acc: 81.500000%
Train Epoch: 11/50 Train_Loss: 0.7260017991065979 Train_acc: 85.500000%
Train Epoch: 12/50 Train_Loss: 0.5629353523254395 Train_acc: 87.750000%
Train Epoch: 13/50 Train_Loss: 0.4164254069328308 Train_acc: 88.937500%
Train Epoch: 14/50 Train_Loss: 0.4018170237541199 Train_acc: 90.375000%
Tr

Train Epoch: 1/50 Train_Loss: 2.363030433654785 Train_acc: 6.500000%
Train Epoch: 2/50 Train_Loss: 2.319702386856079 Train_acc: 6.687500%
Train Epoch: 3/50 Train_Loss: 2.3388330936431885 Train_acc: 6.875000%
Train Epoch: 4/50 Train_Loss: 2.3213326930999756 Train_acc: 8.562500%
Train Epoch: 5/50 Train_Loss: 2.266573190689087 Train_acc: 11.937500%
Train Epoch: 6/50 Train_Loss: 2.1231045722961426 Train_acc: 22.062500%
Train Epoch: 7/50 Train_Loss: 2.0081276893615723 Train_acc: 39.187500%
Train Epoch: 8/50 Train_Loss: 1.7535799741744995 Train_acc: 59.437500%
Train Epoch: 9/50 Train_Loss: 1.4487546682357788 Train_acc: 74.625000%
Train Epoch: 10/50 Train_Loss: 1.0456206798553467 Train_acc: 83.000000%
Train Epoch: 11/50 Train_Loss: 0.8549770712852478 Train_acc: 86.562500%
Train Epoch: 12/50 Train_Loss: 0.5990637540817261 Train_acc: 89.375000%
Train Epoch: 13/50 Train_Loss: 0.3541626036167145 Train_acc: 89.437500%
Train Epoch: 14/50 Train_Loss: 0.3540933430194855 Train_acc: 90.812500%
Train Ep

Train Epoch: 1/50 Train_Loss: 2.288273572921753 Train_acc: 10.437500%
Train Epoch: 2/50 Train_Loss: 2.3077468872070312 Train_acc: 10.125000%
Train Epoch: 3/50 Train_Loss: 2.287614345550537 Train_acc: 10.187500%
Train Epoch: 4/50 Train_Loss: 2.284815788269043 Train_acc: 11.812500%
Train Epoch: 5/50 Train_Loss: 2.201805830001831 Train_acc: 16.312500%
Train Epoch: 6/50 Train_Loss: 2.10396146774292 Train_acc: 26.000000%
Train Epoch: 7/50 Train_Loss: 1.9843674898147583 Train_acc: 43.750000%
Train Epoch: 8/50 Train_Loss: 1.7276921272277832 Train_acc: 63.625000%
Train Epoch: 9/50 Train_Loss: 1.487678050994873 Train_acc: 75.125000%
Train Epoch: 10/50 Train_Loss: 1.0679253339767456 Train_acc: 80.625000%
Train Epoch: 11/50 Train_Loss: 0.7918544411659241 Train_acc: 84.250000%
Train Epoch: 12/50 Train_Loss: 0.6692490577697754 Train_acc: 86.750000%
Train Epoch: 13/50 Train_Loss: 0.5614781975746155 Train_acc: 89.375000%
Train Epoch: 14/50 Train_Loss: 0.4497087001800537 Train_acc: 90.062500%
Train Ep

Train Epoch: 1/50 Train_Loss: 2.303230047225952 Train_acc: 8.875000%
Train Epoch: 2/50 Train_Loss: 2.333534002304077 Train_acc: 9.562500%
Train Epoch: 3/50 Train_Loss: 2.323378562927246 Train_acc: 10.812500%
Train Epoch: 4/50 Train_Loss: 2.2697041034698486 Train_acc: 12.750000%
Train Epoch: 5/50 Train_Loss: 2.182572364807129 Train_acc: 17.750000%
Train Epoch: 6/50 Train_Loss: 2.1278114318847656 Train_acc: 29.062500%
Train Epoch: 7/50 Train_Loss: 2.002603530883789 Train_acc: 43.375000%
Train Epoch: 8/50 Train_Loss: 1.7460830211639404 Train_acc: 62.687500%
Train Epoch: 9/50 Train_Loss: 1.470160961151123 Train_acc: 73.312500%
Train Epoch: 10/50 Train_Loss: 1.1180849075317383 Train_acc: 82.000000%
Train Epoch: 11/50 Train_Loss: 0.7685090899467468 Train_acc: 84.812500%
Train Epoch: 12/50 Train_Loss: 0.7194825410842896 Train_acc: 87.687500%
Train Epoch: 13/50 Train_Loss: 0.44350552558898926 Train_acc: 88.687500%
Train Epoch: 14/50 Train_Loss: 0.40400025248527527 Train_acc: 90.000000%
Train E

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()